# Stock Portfolio with Artificial Intelligence

Project based on You Tube video: https://youtu.be/nXMecZ9oBmQ

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

companies = ["ABEV3", "AZUL4", "BTOW3", "B3SA3", "BBSE3", "BRML3", "BBDC4", "BRAP4", "BBAS3", "BRKM5", "BRFS3", "BPAC11", "CRFB3", "CCRO3", "CMIG4", "HGTX3", "CIEL3", "COGN3", "CPLE6", "CSAN3", "CPFE3", "CVCB3", "CYRE3", "ECOR3", "ELET6", "EMBR3", "ENBR3", "ENGI11", "ENEV3", "EGIE3", "EQTL3", "EZTC3", "FLRY3", "GGBR4", "GOAU4", "GOLL4", "NTCO3", "HAPV3", "HYPE3", "IGTA3", "GNDI3", "ITSA4", "ITUB4", "JBSS3", "JHSF3", "KLBN11", "RENT3", "LCAM3", "LAME4", "LREN3", "MGLU3", "MRFG3", "BEEF3", "MRVE3", "MULT3", "PCAR3", "PETR4", "BRDT3", "PRIO3", "QUAL3", "RADL3", "RAIL3", "SBSP3", "SANB11", "CSNA3", "SULA11", "SUZB3", "TAEE11", "VIVT3", "TIMS3", "TOTS3", "UGPA3", "USIM5", "VALE3", "VVAR3", "WEGE3", "YDUQ3"]

fundamentals = {}

balance_folder = os.path.abspath('balance_sheets/')

for file_name in os.listdir(balance_folder):
    code = file_name[len('balance_'):-len('.xls')]
    if code not in companies:
        continue

    balance = pd.read_excel(os.path.join(balance_folder, file_name), sheet_name = 0)
    balance.iloc[0,0] = code
    balance.columns = balance.iloc[0]
    balance = balance[1:]
    balance = balance.set_index(code)
    
    dre = pd.read_excel(os.path.join(balance_folder, file_name), sheet_name = 1)
    dre.iloc[0,0] = code
    dre.columns = dre.iloc[0]
    dre = dre[1:]
    dre = dre.set_index(code)
    
    fundamentals[code] = pd.concat([balance, dre])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (30309) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (30309) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64274) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64274) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (68412) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (68412) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-z

WARNING *** file size (64336) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64336) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (24947) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (24947) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64290) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64290) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64282) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

WARNING *** file size (64282) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [11]:
quotes_df = pd.read_excel('Quotes.xlsx')
quotes = {}
for code in quotes_df.Company.unique():
    quotes[code] = quotes_df[quotes_df.Company == code]

In [12]:
for comp in companies:
    if quotes[comp].isnull().values.any():
        quotes.pop(comp)
        fundamentals.pop(comp)

companies = list(fundamentals.keys())

In [13]:
for comp in companies:
    table = fundamentals[comp].T
    table.index = pd.to_datetime(table.index, format='%d/%m/%Y')
    
    quotes_table = quotes[comp].set_index('Date')
    quotes_table = quotes_table[['Adj Close']]
    
    table = table.merge(quotes_table, left_index=True, right_index=True)
    table.index.name = comp
    
    fundamentals[comp] = table

In [14]:
standard_cols = list(fundamentals[companies[0]].columns)
for comp in companies:
    if set(standard_cols) != set(fundamentals[comp].columns):
        fundamentals.pop(comp)
companies = list(fundamentals.keys())
print(len(fundamentals))

61


In [15]:
col_text = ';'.join(standard_cols)
updated_cols = []
for col in standard_cols:
    if col_text.count(';' + col + ';') > 1 and col not in updated_cols:
        col_text = col_text.replace(';' + col + ';', ';' + col + '_1;', 1)
        updated_cols.append(col)
standard_cols = [col.replace(' ', '_') for col in col_text.split(';')]

In [18]:
for comp in companies:
    fundamentals[comp].columns = standard_cols

In [19]:
nulls_per_col = dict.fromkeys(standard_cols, 0)
tot_lines = 0

for comp in companies:
    table = fundamentals[comp]
    tot_lines += table.shape[0]
    for col in list(table.columns):
        nulls_per_col[col] += table[col].isnull().sum().astype(int)

for npc in nulls_per_col.items():
    print(npc)
print(f"Total lines = {tot_lines}")

('Ativo_Total', 0)
('Ativo_Circulante', 0)
('Caixa_e_Equivalentes_de_Caixa', 0)
('Aplicações_Financeiras', 0)
('Contas_a_Receber_1', 0)
('Estoques_1', 0)
('Ativos_Biológicos_1', 0)
('Tributos_a_Recuperar', 0)
('Despesas_Antecipadas_1', 0)
('Outros_Ativos_Circulantes', 0)
('Ativo_Realizável_a_Longo_Prazo', 0)
('Aplicações_Financeiras_Avaliadas_a_Valor_Justo', 0)
('Aplicações_Financeiras_Avaliadas_ao_Custo_Amortizado', 0)
('Contas_a_Receber', 0)
('Estoques', 0)
('Ativos_Biológicos', 0)
('Tributos_Diferidos_1', 0)
('Despesas_Antecipadas', 0)
('Créditos_com_Partes_Relacionadas', 0)
('Outros_Ativos_Não_Circulantes', 0)
('Investimentos', 0)
('Imobilizado', 0)
('Intangível', 0)
('Diferido', 0)
('Passivo_Total', 0)
('Passivo_Circulante', 0)
('Obrigações_Sociais_e_Trabalhistas', 0)
('Fornecedores', 0)
('Obrigações_Fiscais', 0)
('Empréstimos_e_Financiamentos_1', 0)
('Passivos_com_Partes_Relacionadas_1', 0)
('Dividendos_e_JCP_a_Pagar', 0)
('Outros_1', 0)
('Provisões_1', 0)
('Passivos_sobre_Ativos

In [22]:
cols_to_remove = [col for col in list(nulls_per_col.keys()) if nulls_per_col[col] > 50]

for comp in fundamentals:
    fundamentals[comp] = fundamentals[comp].drop(cols_to_remove, axis=1)
    fundamentals[comp] = fundamentals[comp].ffill()

In [23]:
print(fundamentals["ABEV3"].shape)

(33, 72)
